In [1]:
from astroquery.gaia import Gaia
from astropy import units as u
from astropy.constants import h, c
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [70]:
# Constants
fd_unit = u.J/u.m**2/u.s/u.nm
d_filters = {'Band':['G','BP','RP'],'Wavelength':[639.07*u.nm,518.26*u.nm,782.51*u.nm],'Bandpass':[454.82*u.nm,265.9*u.nm,292.75*u.nm],'clam':[1.346e-21*fd_unit,3.009e-21*fd_unit,1.638e-21*fd_unit]} 
filters = pd.DataFrame(data=d_filters)

ccd_width_px = 4656
ccd_height_px = 3520
ccd_px_side_length_micron = 3.8 * u.micron
ccd_width_micron = ccd_width_px * ccd_px_side_length_micron
ccd_height_micron = ccd_height_px * ccd_px_side_length_micron
plate_scale_arcsec_micron = 0.11 * u.arcsec
ccd_width_range_arcsec = ccd_width_micron * plate_scale_arcsec_micron
ccd_height_range_arcsec = ccd_height_micron * plate_scale_arcsec_micron
ccd_width_range_deg = u.arcsec.to(u.deg, value=ccd_width_range_arcsec)
ccd_height_range_deg = u.arcsec.to(u.deg, value=ccd_height_range_arcsec)
mast_collecting_area_m2 = 0.09 * np.pi * 0.5 * u.m**2
gaia_collecting_area_m2 = 0.7 * u.m**2
filters

,Band,Wavelength,Bandpass,clam
0,G,639.07 nm,454.82 nm,1.346e-21 J / (m2 nm s)
1,BP,518.26 nm,265.9 nm,3.009e-21 J / (m2 nm s)
2,RP,782.51 nm,292.75 nm,1.638e-21 J / (m2 nm s)


In [51]:
ra_widget = widgets.BoundedFloatText(
    value=279.235,
    min=0,
    max=360,
    step=0.001,
    description='RA:'
)
dec_widget = widgets.BoundedFloatText(
    value=38.783,
    min=-90,
    max=90,
    step=0.001,
    description='DEC:'
)
max_mag_widget = widgets.FloatText(
    value=18,
    step=0.01,
    description='Max Mag:'
)
display(ra_widget)
display(dec_widget)
display(max_mag_widget)

BoundedFloatText(value=279.235, description='RA:', max=360.0, step=0.001)

BoundedFloatText(value=38.783, description='DEC:', max=90.0, min=-90.0, step=0.001)

FloatText(value=18.0, description='Max Mag:', step=0.01)

In [52]:
def get_entries(max_mag, ra, dec, ccd_width_range_deg, ccd_height_range_deg):
    query = '''SELECT TOP 100 RA, DEC,
PHOT_G_MEAN_FLUX AS PHOT_G_MEAN_FLUX_GAIA, PHOT_G_MEAN_FLUX_ERROR AS PHOT_G_MEAN_FLUX_ERROR_GAIA,
PHOT_BP_MEAN_FLUX AS PHOT_BP_MEAN_FLUX_GAIA, PHOT_BP_MEAN_FLUX_ERROR AS PHOT_BP_MEAN_FLUX_ERROR_GAIA,
PHOT_RP_MEAN_FLUX AS PHOT_RP_MEAN_FLUX_GAIA, PHOT_RP_MEAN_FLUX_ERROR AS PHOT_RP_MEAN_FLUX_ERROR_GAIA
FROM gaiadr3.gaia_source
WHERE (ra BETWEEN {ra_min} AND {ra_max})
AND (dec BETWEEN {dec_min} AND {dec_max})
AND phot_g_mean_mag < {max_mag}
'''.format(
        max_mag=max_mag,
        ra_min=ra-(ccd_width_range_deg.value/2), ra_max=ra+(ccd_width_range_deg.value/2),
        dec_min=dec-(ccd_height_range_deg.value/2), dec_max=dec+(ccd_height_range_deg.value/2)
    )
    job = Gaia.launch_job(query)
    r = job.get_results()
    return r

In [68]:
def convert_to_flux_density(table):
    # gets flux mean, flux error in electrons/sec (as given in GAIA_SOURCE) and name of band {'G','BP,'RP'}
    # returns flux density + uncertainty in watt/m^2/nm
    for filter in d_filters['Band']:
        table['flux_' + filter + '_density_gaia'] = filters[filters['Band'] == filter]['clam'].iloc[0] * table['phot_' + str.lower(filter) + '_mean_flux_gaia'].value # convert to incoming flux density
        table['flux_' + filter + '_density_error_gaia'] =  filters[filters['Band'] == filter]['clam'].iloc[0] * table['phot_' + str.lower(filter) + '_mean_flux_error_gaia'].value

In [54]:
def calculate_mast_qe(flux_density_inc_mast):
    return flux_density_inc_mast

In [74]:
def convert_gaia_to_mast(table):
    for filter in d_filters['Band']:
        lam0 = filters[filters.Band == filter].iloc[0]['Wavelength']
        bandpass = filters[filters.Band == filter].iloc[0]['Bandpass']
        gaia_mast_attenuation = 0.5
        flux_density_inc_mast =  gaia_mast_attenuation * table['flux_' + filter + '_density_gaia']
        flux_meas_mast = calculate_mast_qe(flux_density_inc_mast) * bandpass
        power_meas_mast = flux_meas_mast * mast_collecting_area_m2
        table['phot_' + filter + '_flux_mast'] = (power_meas_mast * lam0.to(u.m) / (h * c)) * u.electron 

        flux_density_inc_error_mast =  gaia_mast_attenuation * table['flux_' + filter + '_density_error_gaia']
        flux_meas_error_mast = calculate_mast_qe(flux_density_inc_error_mast) * bandpass
        power_meas_error_mast = flux_meas_error_mast * mast_collecting_area_m2
        table['phot_' + filter + '_flux_rel_error_mast'] = np.sqrt(((power_meas_error_mast * lam0.to(u.m) / (h * c)).value)**2 + ((power_meas_mast * bandpass.to(u.m) / (2 * h * c)).value)**2) / table['phot_' + filter + '_flux_mast'].value * 100 * u.percent

In [75]:
entries = get_entries(max_mag_widget.value, ra_widget.value, dec_widget.value, ccd_width_range_deg, ccd_height_range_deg)
convert_to_flux_density(entries)
convert_gaia_to_mast(entries)
entries[0]

ra,dec,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,flux_G_density_gaia,flux_G_density_error_gaia,flux_BP_density_gaia,flux_BP_density_error_gaia,flux_RP_density_gaia,flux_RP_density_error_gaia,phot_G_flux_mast,phot_G_flux_rel_error_mast,phot_BP_flux_mast,phot_BP_flux_rel_error_mast,phot_RP_flux_mast,phot_RP_flux_rel_error_mast
deg,deg,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),electron / s,%,electron / s,%,electron / s,%
float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64
278.96494437636767,38.658773196656206,4302.854510364426,2.3290374,2296.7434898592805,13.387459,2837.5006055939084,8.756109,5.7916421709505165e-18,3.1348843e-21,6.910901160986575e-18,4.0282862e-20,4.647825991962822e-18,1.4342507e-20,599.0255752086268,35.58456242435562,338.88813849490157,25.659768388497156,378.8718327079242,18.708376388723764
